## Loading molecule from QCA and calculating the MM energy with a SMIRNOFF-format forcefield

## Energy decomposition for an openmm system



In [1]:
from openforcefield.topology import Molecule, Topology
from openforcefield.typing.engines.smirnoff.forcefield import ForceField
from openforcefield.utils import get_data_file_path
from simtk import openmm, unit
import numpy as np

### Define utility function to get energy of an OpenMM system \
Force groups in openmm \
0  - 'HarmonicAngleForce' \
0 - 'HarmonicBondForce' \
11  - 'NonbondedForce'\
0  - 'PeriodicTorsionForce'

Not using the other functions forcegroupify() and getEnergyDecomposition()

In [2]:
def forcegroupify(system):
    forcegroups = {}
    for i in range(system.getNumForces()):
        force = system.getForce(i)
        force.setForceGroup(i)
        forcegroups[force] = i
    return forcegroups

def getEnergyDecomposition(context, forcegroups):
    energies = {}
    for f, i in forcegroups.items():
        print(f, i)
        energies[f] = context.getState(getEnergy=True, groups=2**i).getPotentialEnergy()
    return energies


def get_energy(system, positions, groups):
    """
    Return the potential energy.

    Parameters
    ----------
    system : simtk.openmm.System
        The system to check
    positions : simtk.unit.Quantity of dimension (natoms,3) with units of length
        The positions to use
    groups : set of integers that represent forcegroups (setting Nonbonded as non-zero). Eg. {0, 11}
    
    Returns
    ---------
    energy
    """

    integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
    context = openmm.Context(system, integrator)
    context.setPositions(positions)
    for force in system.getForces():
        if force.__class__.__name__ == 'NonbondedForce':
            force.setForceGroup(11)    
#         print(force, force.getForceGroup())
    state = context.getState(getEnergy=True, groups=groups)
    energy = state.getPotentialEnergy().in_units_of(unit.kilocalorie_per_mole)
    return energy

## Load a molecule with positions, and evaluate its energy, evaluate the Nonbonded term

In this example, we load a single molecule with geometry information, parameterize it using the "Parsley" (`openff-1.3.0`) forcefield, and evaluate its energy. 



Load a molecule

In [3]:
import qcportal as ptl
client = ptl.FractalClient()
ds = client.get_collection("TorsionDriveDataset", "OpenFF Rowley Biaryl v1.0")
ds.status("default", status="COMPLETE")
benchmark_smiles="lim_mobley_parsley_benchmark.smi"
with open(benchmark_smiles) as f:
    bm_smiles = f.readlines()
bm_mols = [Molecule.from_smiles(smiles) for smiles in bm_smiles]
ff = ForceField('openff_unconstrained-1.3.0.offxml')

for i in range(1): #ds.df.size):
    if ds.df.iloc[i, 0].status == "COMPLETE":
        smiles = ds.df.index[i]
        mapped_smiles = ds.get_entry(smiles).attributes[
            "canonical_isomeric_explicit_hydrogen_mapped_smiles"
        ]
        mol1 = Molecule.from_mapped_smiles(mapped_smiles)
        not_identical = True
        for mol in bm_mols:
            isomorphic,atom_map = Molecule.are_isomorphic(mol1, 
                                              mol,
                                              return_atom_map=False,
                                              aromatic_matching=False,
                                              formal_charge_matching=False,
                                              bond_order_matching=False,
                                              atom_stereochemistry_matching=False,
                                              bond_stereochemistry_matching=False,
                                                      )
            if(isomorphic):
                not_identical = False
                overlaps += 1
                entry = ds.get_entry(smiles)
                tdr_id = entry.object_map['default']
                qca_entries.append(tdr_id)
                break
        if(not_identical): 
            td = ds.df.iloc[i, 0]
            mol_dict = td.get_final_molecules()
            energies_dict = td.get_final_energies()
            max_idx = max(energies_dict, key=energies_dict.get)
            min_idx = min(energies_dict, key=energies_dict.get)
            print(max_idx, min_idx)
            mol = Molecule.from_qcschema(ds.get_entry(smiles))
            mol_dict = td.get_final_molecules()
            min_positions = mol_dict[min_idx].geometry * unit.angstrom
            max_positions = mol_dict[max_idx].geometry * unit.angstrom
            topology = mol.to_topology()
            system = ff.create_openmm_system(topology)
            nonbonded_energy_min_pos = get_energy(system, min_positions, {11})
            nonbonded_energy_max_pos = get_energy(system, max_positions, {11})
            total_energy_min_pos = get_energy(system, min_positions, {0,11})
            total_energy_max_pos = get_energy(system, max_positions, {0,11})
            nonbonded_energy_diff = nonbonded_energy_max_pos - nonbonded_energy_min_pos
            total_energy_diff = total_energy_max_pos - total_energy_min_pos
            qm_energy_diff = 627.509 *(energies_dict[max_idx] - energies_dict[min_idx])
            print("TD record:", td.id,", nonbonded energy diff:", nonbonded_energy_diff, ", approx. MM barrier:", total_energy_diff, ", QM barrier:", qm_energy_diff)

(90,) (15,)
TD record: 21272352 , nonbonded energy diff: 0.20875432511871495 kcal/mol , approx. MM barrier: 15.75993197410935 kcal/mol , QM barrier: 4.430783246457765


#### Still something wrong with the MM energy calculation in above block, from the QM-MM plots the MM barrier in this case (TD record: 21272352) should be almost equal to QM barrier value (as calculated using Yudong's script)